In [77]:
import sys
sys.path.append("..")
%reload_ext autoreload
%autoreload 2
import os
import pandas as pd
import pyreadstat as pyr
import json
import numpy as np
from spss_import import read_sav 
pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None

In [135]:
spssfile = f"files/SPSS_Example2.sav"
df, df_meta = read_sav(spssfile)
df.head()

,RID,MARST,PWT,testvar
0,10000001,3,537,1
1,10000002,1,231,1
2,10000003,2,599,7
3,10000004,1,4003,8
4,10000005,4,598,9


In [122]:
spssfile2 = f"files/ESS-Data-Wizard-subset-2023-10-21.dta"
df2, df2_meta = read_sav(spssfile2)
df2.head()

,name,essround,edition,proddate,idno,cntry,dweight,pspwght,pweight,anweight,tvtot,tvpol,ppltrst,pplfair,pplhlp
0,ESS6e02_5,6,2.5,19.01.2023,1,AL,0.775069,0.536221,0.186008,0.099742,7,6,5,3,0
1,ESS6e02_5,6,2.5,19.01.2023,2,AL,1.354429,0.65952,0.186008,0.122676,7,2,0,b,0
2,ESS6e02_5,6,2.5,19.01.2023,3,AL,2.933552,1.81561,0.186008,0.337718,4,0,5,5,4
3,ESS6e02_5,6,2.5,19.01.2023,4,AL,0.968836,0.534046,0.186008,0.099337,0,a,10,5,2
4,ESS6e02_5,6,2.5,19.01.2023,5,AL,1.325001,1.880963,0.186008,0.349874,6,2,5,6,7


In [106]:
def create_variable_view(df_meta):
    # Extract the attributes from df_meta
    label = df_meta.column_names_to_labels
    values = df_meta.variable_value_labels
    missing = df_meta.missing_ranges
    format = df_meta.original_variable_types
    measure = df_meta.variable_measure

    # Convert dictionaries into individual dataframes
    df_label = pd.DataFrame(list(label.items()), columns=['name', 'label'])
    df_format = pd.DataFrame(list(format.items()), columns=['name', 'format'])
    df_measure = pd.DataFrame(list(measure.items()), columns=['name', 'measure'])

    # For values and missing, we will handle them differently since they have dictionaries/lists inside
    df_values_list = [{'name': k, 'values': str(v)} for k, v in values.items()]  # Convert values to string
    df_values = pd.DataFrame(df_values_list)

    df_missing_list = [{'name': k, 'missing': str(v)} for k, v in missing.items()]  # Convert missing values to string
    df_missing = pd.DataFrame(df_missing_list)

    # Merge dataframes on the 'name' column
    variable_view = df_label.merge(df_values, on='name', how='outer') \
        .merge(df_missing, on='name', how='outer') \
        .merge(df_format, on='name', how='outer') \
        .merge(df_measure, on='name', how='outer')

    return variable_view
create_variable_view(df_meta)

,name,label,values,missing,format,measure
0,name,Title of dataset,NaN,NaN,A10,scale
1,essround,ESS round,NaN,NaN,F2.0,scale
2,edition,Edition,NaN,NaN,A3,scale
3,proddate,Production date,NaN,NaN,A10,scale
4,idno,Respondent's identification number,NaN,NaN,F5.0,nominal
5,cntry,Country,"{'AL': 'Albania', 'AT': 'Austria', 'BE': 'Belg...",NaN,A2,nominal
6,dweight,Design weight,NaN,NaN,F3.2,scale
7,pspwght,Post-stratification weight including design we...,NaN,NaN,F3.2,scale
8,pweight,Population size weight (must be combined with ...,NaN,NaN,F3.2,scale
9,anweight,Analysis weight,NaN,NaN,F3.2,scale


In [112]:
df2_meta.variable_value_labels['cptppola']

{1: 'Not at all confident',
 2: 'A little confident',
 3: 'Quite confident',
 4: 'Very confident',
 5: 'Completely confident',
 'a': 'Refusal',
 'b': "Don't know",
 'c': 'No answer'}

In [114]:
def create_variable_view2(df_meta):
    # Extract the attributes from df_meta
    label = df_meta.column_names_to_labels
    values = df_meta.variable_value_labels
    
    # Convert user-defined missing values to the desired format
    missing = {}
    for key, vals in df_meta.missing_user_values.items():
        missing[key] = [{"lo": val, "hi": val} for val in vals]

    format = df_meta.original_variable_types
    measure = df_meta.variable_measure

    # Convert dictionaries into individual dataframes
    df_label = pd.DataFrame(list(label.items()), columns=['name', 'label'])
    df_format = pd.DataFrame(list(format.items()), columns=['name', 'format'])
    df_measure = pd.DataFrame(list(measure.items()), columns=['name', 'measure'])

    # For values and missing, we will handle them differently since they have dictionaries/lists inside
    df_values_list = [{'name': k, 'values': str(v)} for k, v in values.items()]  # Convert values to string
    df_values = pd.DataFrame(df_values_list)

    df_missing_list = [{'name': k, 'missing': str(v)} for k, v in missing.items()]  # Convert missing values to string
    df_missing = pd.DataFrame(df_missing_list)

    # Merge dataframes on the 'name' column
    variable_view = df_label.merge(df_values, on='name', how='outer') \
        .merge(df_missing, on='name', how='outer') \
        .merge(df_format, on='name', how='outer') \
        .merge(df_measure, on='name', how='outer')

    return variable_view
create_variable_view2(df2_meta)

,name,label,values,missing,format,measure
0,name,Title of dataset,NaN,NaN,%10s,unknown
1,essround,ESS round,NaN,NaN,%4.0g,unknown
2,edition,Edition,NaN,NaN,%3s,unknown
3,proddate,Production date,NaN,NaN,%10s,unknown
4,idno,Respondent's identification number,NaN,NaN,%7.0g,unknown
5,cntry,Country,NaN,NaN,%2s,unknown
6,dweight,Design weight,NaN,NaN,%3.2g,unknown
7,pspwght,Post-stratification weight including design we...,NaN,NaN,%3.2g,unknown
8,pweight,Population size weight (must be combined with ...,NaN,NaN,%3.2g,unknown
9,anweight,Analysis weight,NaN,NaN,%3.2g,unknown


In [ ]:
Label = df_meta.column_names_to_labels
Values = df_meta.variable_value_labels
Missing = df_meta.missing_ranges
Format = df_meta.original_variable_types
Measure = df_meta.variable_measure

In [96]:
df_meta.missing_ranges

{'panpriph': [{'lo': 66.0, 'hi': 99.0}],
 'panmonpb': [{'lo': 66.0, 'hi': 99.0}],
 'panfolru': [{'lo': 77.0, 'hi': 77.0},
  {'lo': 88.0, 'hi': 88.0},
  {'lo': 99.0, 'hi': 99.0}],
 'panclobo': [{'lo': 77.0, 'hi': 77.0},
  {'lo': 88.0, 'hi': 88.0},
  {'lo': 99.0, 'hi': 99.0}],
 'panresmo': [{'lo': 77.0, 'hi': 77.0},
  {'lo': 88.0, 'hi': 88.0},
  {'lo': 99.0, 'hi': 99.0}],
 'nwspol': [{'lo': 7777.0, 'hi': 7777.0},
  {'lo': 8888.0, 'hi': 8888.0},
  {'lo': 9999.0, 'hi': 9999.0}],
 'netusoft': [{'lo': 7.0, 'hi': 7.0},
  {'lo': 8.0, 'hi': 8.0},
  {'lo': 9.0, 'hi': 9.0}],
 'netustm': [{'lo': 7777.0, 'hi': 9999.0}, {'lo': 6666.0, 'hi': 6666.0}],
 'ppltrst': [{'lo': 77.0, 'hi': 77.0},
  {'lo': 88.0, 'hi': 88.0},
  {'lo': 99.0, 'hi': 99.0}],
 'pplfair': [{'lo': 77.0, 'hi': 77.0},
  {'lo': 88.0, 'hi': 88.0},
  {'lo': 99.0, 'hi': 99.0}],
 'pplhlp': [{'lo': 77.0, 'hi': 77.0},
  {'lo': 88.0, 'hi': 88.0},
  {'lo': 99.0, 'hi': 99.0}],
 'polintr': [{'lo': 7.0, 'hi': 7.0},
  {'lo': 8.0, 'hi': 8.0},
  {'l

In [ ]:
def generate_InstanceVariable(df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):
        elements = {
            "@id": f"#{variable}",
            "@type": "InstanceVariable",
            "name": variable,
            "displayLabel": df_meta.column_labels[idx],
            "hasIntendedDataType": df_meta.original_variable_types[variable]
        }

        # Check if variable is in keys and if so, add an extra key-value pair
        if variable in df_meta.variable_value_labels:
            elements['takesSubstantiveConceptsFrom'] = f"#substantiveConceptualDomain-{variable}"
        
        if variable in df_meta.missing_ranges:
            elements['takesSentinelConceptsFrom'] = f"#sentinelConceptualDomain-{variable}"

        json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# MeasureComponent
def generate_MeasureComponent(df_meta):
    json_ld_data = []
    for x, variable in enumerate(df_meta.column_names[1:]):
        elements = {
            "@id": f"#measureComponent-{variable}",
            "@type": "MeasureComponent",
            "isDefinedBy": f"#{variable}"
        }
        json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# MeasureComponent2
def generate_MeasureComponent2(df_meta, varlist=None):
    json_ld_data = []
    for x, variable in enumerate(df_meta.column_names):
        if variable not in varlist:
            elements = {
                "@id": f"#measureComponent-{variable}",
                "@type": "MeasureComponent",
                "isDefinedBy": f"#{variable}"
            }
            json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# IdentifierComponent
def generate_IdentifierComponent(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#identifierComponent-{df_meta.column_names[0]}",
        "@type": "IdentifierComponent",
        "isDefinedBy": f"#{df_meta.column_names[0]}"
    }
    json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# IdentifierComponent2
def generate_IdentifierComponent2(df_meta, varlist=None):
    json_ld_data = []
    for x, variable in enumerate(df_meta.column_names):
        if variable in varlist:
            elements = {
                "@id": f"#identifierComponent-{variable}",
                "@type": "IdentifierComponent",
                "isDefinedBy": f"#{variable}"
            }
            json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# logicalRecord
def generate_LogicalRecord(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#logicalRecord",
        "@type": "LogicalRecord",
        "organizes": f"#wideDataSet"
    }
    has = []    
    for x, variable in enumerate(df_meta.column_names):
        has.append(f"#{variable}")
    elements['has'] = has
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# PhysicalDataset
def generate_PhysicalDataset(df_meta, spssfile):
    json_ld_data = []
    elements = {
        "@id": f"#physicalDataset",
        "@type": "PhysicalDataset",
        "formats": "#dataStore",
        "physicalFileName": spssfile
    }
    has = ["#physicalRecordSegment"]
    elements['has'] = has    
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# DataStore
def generate_DataStore(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#dataStore",
        "@type": "DataStore",
        "recordCount": df_meta.number_rows
    }
    has = ["#logicalRecord"]
    elements['has'] = has   
    
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# WideDataSet
def generate_WideDataSet(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#wideDataSet",
        "@type": "WideDataSet",
        "isStructuredBy": "#wideDataStructure"
    }
    
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# WideDataStructure
def generate_WideDataStructure(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#wideDataStructure",
        "@type": "WideDataStructure",
    }
    has = ["#primaryKey", f"#identifierComponent-{df_meta.column_names[0]}"]
    
    for x, variable in enumerate(df_meta.column_names[1:]):
        has.append(f"#measureComponent-{variable}")
    elements['has'] = has
        
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# WideDataStructure2
def generate_WideDataStructure2(df_meta, varlist=None):
    json_ld_data = []
    elements = {
        "@id": f"#wideDataStructure",
        "@type": "WideDataStructure",
    }
    has = ["#primaryKey"]
    
    for x, variable in enumerate(df_meta.column_names):
        if variable in varlist:
            has.append(f"#identifierComponent-{variable}")
        else:
            has.append(f"#measureComponent-{variable}")
    elements['has'] = has
        
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# PrimaryKey
def generate_PrimaryKey(df_meta):
    json_ld_data = []
    elements = {
        "@id": "#primaryKey",
        "@type": "PrimaryKey",
        "isComposedOf": "#primaryKeyComponent"
    }
        
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# PrimaryKeyComponent
def generate_PrimaryKeyComponent(df_meta):
    json_ld_data = []
    elements = {
        "@id": "#primaryKeyComponent",
        "@type": "PrimaryKeyComponent",
        "correspondsTo": f"#identifierComponent-{df_meta.column_names[0]}"
    }
        
    json_ld_data.append(elements)
    return json_ld_data


In [ ]:
# PrimaryKeyComponent2
def generate_PrimaryKeyComponent2(df_meta, varlist=None):
    json_ld_data = []
    elements = {
        "@id": "#primaryKeyComponent",
        "@type": "PrimaryKeyComponent",
    }
    has = []    
    for variable in varlist:
        has.append(f"#identifierComponent-{variable}")
    
    elements['correspondsTo'] = has      
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# PhysicalRecordSegment
def generate_PhysicalRecordSegment(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#physicalRecordSegment",
        "@type": "PhysicalRecordSegment",
        "mapsTo": "#logicalRecord",
    }
    has = ["#physicalSegmentLayout"]
    elements['has'] = has    
    
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# PhysicalSegmentLayout
def generate_PhysicalSegmentLayout(df_meta):
    json_ld_data = []
    elements = {
        "@id": f"#physicalSegmentLayout",
        "@type": "PhysicalSegmentLayout",
        "formats": "#logicalRecord",
        "isDelimited": "false",
        "delimiter": "",
    }
    has = []    
    for x, variable in enumerate(df_meta.column_names):
        has.append(f"#valueMapping-{variable}")
        has.append(f"#valueMappingPosition-{variable}")
    elements['has'] = has    
    json_ld_data.append(elements)
    return json_ld_data

In [ ]:
# ValueMapping
def generate_ValueMapping(df, df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):
        elements = {
            "@id": f"#valueMapping-{variable}",
            "@type": "ValueMapping",
        }
        datapoints = []
        for i, x in enumerate(df[variable]):
            datapoints.append(f"#{variable}-dataPoint-{i}")
        elements['formats'] = datapoints

        json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# ValueMappingPosition
def generate_ValueMappingPosition(df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for idx, variable in enumerate(df_meta.column_names):
        elements = {
            "@id": f"#valueMappingPosition-{variable}",
            "@type": "ValueMappingPosition",
            "value": idx,
            "indexes": f"#valueMapping-{variable}"
        }
        json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# DataPoint
def generate_DataPoint(df, df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for variable in (df_meta.column_names):
        for idx, value in enumerate(df[variable]):
            elements = {
                "@id": f"#{variable}-dataPoint-{idx}",
                "@type": "DataPoint",
                "isDescribedBy": f"#{variable}-instanceValue-{idx}"
            }

            json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# DataPointPosition
def generate_DataPointPosition(df, df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for variable in (df_meta.column_names):
        for idx, value in enumerate(df[variable]):
            elements = {
                "@id": f"#{variable}-dataPointPosition-{idx}",
                "@type": "DataPointPosition",
                "value": idx,
                "indexes": f"#{variable}-dataPoint-{idx}"
            }

            json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# SubstantiveValueDomain
def generate_SubstantiveValueDomain(df_meta):
    json_ld_data = []
    for x, variable in enumerate(df_meta.variable_value_labels):
        elements = {
            "@id": f"#substantiveValueDomain-{variable}",
            "@type": "SubstantiveValueDomain",
            "takesValuesFrom": f"#codelist.{variable}"
        }
        json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# InstanceValue
def generate_InstanceValue(df, df_meta):
    json_ld_data = []

    # Iterate through column names and associated index
    for variable in (df_meta.column_names):
        for idx, value in enumerate(df[variable]):
            elements = {
                "@id": f"#{variable}-instanceValue-{idx}",
                "@type": "InstanceValue",
                "content": value,
                "isStoredIn": f"#{variable}-dataPoint-{idx}"
            }

            json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# Codelist
def generate_Codelist(df_meta):
    json_ld_data = []
    for x, variable in enumerate(df_meta.variable_value_labels.items()):
        elements = {
            "@id": f"#codelist-{variable[x]}",
            "@type": "Codelist",
        }
        has = []
        your_dict = variable[1]
        # Loop through the dictionary and extract the keys
        for key in your_dict.keys():
            codes = {
                "@id": f"#code-{key}-{variable[x]}"
            }
            has.append(codes)
        elements['has'] = has    

        json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# Code
def generate_Code(df_meta):
    json_ld_data = []
    for x, variable in enumerate(df_meta.variable_value_labels.items()):
        your_dict = variable[1]
        # Loop through the dictionary and extract the keys
        for key, value in your_dict.items():
            elements = {
                "@id": f"#code-{key}-{variable[x]}",
                "@type": "Code",
            }
            has = []
            codes = {
                "@id": f"#{key}"
            }
            has.append(codes)
            elements['denotes'] = has

            has = []
            codes = {
                "@id": f"#{value}"
            }
            has.append(codes)
            elements['uses'] = has 

            json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# SubstantiveConceptualDomain
def generate_SubstantiveConceptualDomain(df_meta):
    json_ld_data = []
    for variable in (df_meta.variable_value_labels):
        elements = {
            "@id": f"#substantiveConceptualDomain-{variable}",
            "@type": "SubstantiveConceptualDomain",
            "takesConceptsFrom": f"#substantiveConceptScheme-{variable}"
        }
        json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# SubstantiveConceptScheme
def generate_SubstantiveConceptScheme(df_meta):
    json_ld_data = []

    for variable_name, values_dict in df_meta.variable_value_labels.items():
        elements = {
            "@id": f"#substantiveConceptScheme-{variable_name}",
            "@type": "skos:ConceptScheme",
        }

        has_top_concept = []
        
        # Check if variable_name is in missing_ranges
        if variable_name in df_meta.missing_ranges:
            excluded_ranges = set()
            for dict_range in df_meta.missing_ranges[variable_name]:
                lo_is_numeric = isinstance(dict_range['lo'], (int, float)) or (
                    isinstance(dict_range['lo'], str) and dict_range['lo'].isnumeric()
                )
                hi_is_numeric = isinstance(dict_range['hi'], (int, float)) or (
                    isinstance(dict_range['hi'], str) and dict_range['hi'].isnumeric()
                )

                if lo_is_numeric and hi_is_numeric:
                    # Case: 'lo' and 'hi' can be converted to int
                    excluded_ranges.update(
                        range(int(float(dict_range['lo'])), int(float(dict_range['hi'])) + 1)
                    )
                elif isinstance(dict_range['lo'], str):
                    # Case: 'lo' is a string that is not numeric
                    excluded_ranges.add(dict_range['lo'])
                else:
                    print(f"Warning: Unsupported 'lo' value: {dict_range['lo']}")

            # Use list comprehension to generate the hasTopConcept list
            has_top_concept = [
                f"#{variable_name}-concept-{value}"
                for value in values_dict.keys()
                if not is_value_in_excluded_ranges(value, excluded_ranges)
            ]
        else:
            has_top_concept = [
                f"#{variable_name}-concept-{value}"
                for value in values_dict.keys()
            ]
        
        # Add the hasTopConcept list to elements
        elements['skos:hasTopConcept'] = has_top_concept
        json_ld_data.append(elements)
    
        
    return json_ld_data

def is_value_in_excluded_ranges(value, excluded_ranges):
    try:
        return int(value) in excluded_ranges
    except ValueError:
        # value is not a number, skip it
        return False

In [ ]:
# SentinelConceptualDomain
def generate_ValueAndConceptDescription(df_meta):
    json_ld_data = []
    for variable, value in df_meta.missing_ranges.items():

        elements = {
            "@id": f"#valueAndConceptDescription-{variable}",
            "@type": "ValueAndConceptDescription",
            "isDescribedBy": f"{value}",
        }
            
        json_ld_data.append(elements)

    return json_ld_data

In [ ]:
# SentinelConceptualDomain
def generate_SentinelConceptualDomain(df_meta):
    json_ld_data = []
    for variable in df_meta.missing_ranges:

        elements = {
            "@id": f"#sentinelConceptualDomain-{variable}",
            "@type": "SentinelConceptualDomain",
            "isDescribedBy": f"#valueAndConceptDescription-{variable}",
        }

        if variable in df_meta.variable_value_labels.keys():
            elements["takesConceptsFrom"] = f"#sentinelConceptScheme-{variable}"
            
        json_ld_data.append(elements)

    return json_ld_data

In [ ]:
def generate_SentinelConceptScheme(df_meta):
    json_ld_data = []

    for variable_name, values_dict in df_meta.variable_value_labels.items():
        elements = {
            "@id": f"#sentinelConceptScheme-{variable_name}",
            "@type": "skos:ConceptScheme",
        }

        has_top_concept = []
        
        # Check if variable_name is in missing_ranges
        if variable_name in df_meta.missing_ranges:
            excluded_ranges = set()
            for dict_range in df_meta.missing_ranges[variable_name]:
                lo_is_numeric = isinstance(dict_range['lo'], (int, float)) or (
                    isinstance(dict_range['lo'], str) and dict_range['lo'].isnumeric()
                )
                hi_is_numeric = isinstance(dict_range['hi'], (int, float)) or (
                    isinstance(dict_range['hi'], str) and dict_range['hi'].isnumeric()
                )

                if lo_is_numeric and hi_is_numeric:
                    # Case: 'lo' and 'hi' can be converted to int
                    excluded_ranges.update(
                        range(int(float(dict_range['lo'])), int(float(dict_range['hi'])) + 1)
                    )
                elif isinstance(dict_range['lo'], str):
                    # Case: 'lo' is a string that is not numeric
                    excluded_ranges.add(dict_range['lo'])
                else:
                    print(f"Warning: Unsupported 'lo' value: {dict_range['lo']}")

            # Use list comprehension to generate the hasTopConcept list
            has_top_concept = [
                f"#{variable_name}-concept-{value}"
                for value in values_dict.keys()
                if is_value_in_excluded_ranges(value, excluded_ranges)
            ]
        
        # Add the hasTopConcept list to elements
        elements['skos:hasTopConcept'] = has_top_concept
        
        json_ld_data.append(elements)

    return json_ld_data

In [148]:
# Concept
def generate_Concept(df_meta):
    def is_value_in_excluded_ranges(value, excluded_ranges):
        # Ensure value is of the correct type
        if all(isinstance(i, float) for i in excluded_ranges):
            try:
                value = float(value)
            except ValueError:
                return False
        return value in excluded_ranges

    json_ld_data = []

    # Convert user-defined missing values to the desired format
    missing = df_meta.missing_ranges
    if len(missing) == 0:
        missing = {}
        for key, vals in df_meta.missing_user_values.items():
            missing[key] = [{"lo": val, "hi": val} for val in vals]

    for variable_name, values_dict in df_meta.variable_value_labels.items():
        # Check if variable_name is in missing and, if so, generate the excluded_ranges
        excluded_ranges = set()
        if variable_name in missing:
            for dict_range in missing[variable_name]:
                excluded_ranges.add(dict_range['lo'])
                if dict_range['lo'] != dict_range['hi']:
                    excluded_ranges.add(dict_range['hi'])

        # Iterate through values_dict and create elements, taking into account excluded_keys
        for key, value in values_dict.items():
            elements = {
                "@id": f"#{variable_name}-concept-{key}",
                "@type": "skos:Concept",
                "notation": key,
                "prefLabel": f"{value}",
            }

            # Add the inScheme key to elements based on whether the key is in excluded_ranges
            if is_value_in_excluded_ranges(key, excluded_ranges):
                elements['inScheme'] = f"#sentinelConceptScheme-{variable_name}"
            else:
                elements['inScheme'] = f"#substantiveConceptScheme-{variable_name}"

            # Append elements to json_ld_data inside the loop
            json_ld_data.append(elements)

    return json_ld_data
generate_Concept(df_meta)

[{'@id': '#MARST-concept-1.0',
  '@type': 'skos:Concept',
  'notation': 1.0,
  'prefLabel': 'Single',
  'inScheme': '#substantiveConceptScheme-MARST'},
 {'@id': '#MARST-concept-2.0',
  '@type': 'skos:Concept',
  'notation': 2.0,
  'prefLabel': 'Married',
  'inScheme': '#substantiveConceptScheme-MARST'},
 {'@id': '#MARST-concept-3.0',
  '@type': 'skos:Concept',
  'notation': 3.0,
  'prefLabel': 'In a civil partnership',
  'inScheme': '#substantiveConceptScheme-MARST'},
 {'@id': '#MARST-concept-7.0',
  '@type': 'skos:Concept',
  'notation': 7.0,
  'prefLabel': 'missing1',
  'inScheme': '#sentinelConceptScheme-MARST'},
 {'@id': '#MARST-concept-8.0',
  '@type': 'skos:Concept',
  'notation': 8.0,
  'prefLabel': 'missing2',
  'inScheme': '#sentinelConceptScheme-MARST'},
 {'@id': '#MARST-concept-9.0',
  '@type': 'skos:Concept',
  'notation': 9.0,
  'prefLabel': 'missing3',
  'inScheme': '#sentinelConceptScheme-MARST'},
 {'@id': '#testvar-concept-1',
  '@type': 'skos:Concept',
  'notation': '1

In [141]:
df2_meta.missing_user_values

{'pplfair': ['b'],
 'tvpol': ['a'],
 'ppltrst': ['b'],
 'pplhlp': ['a', 'b'],
 'tvtot': ['b']}

In [146]:
# Generate JSON-LD
InstanceVariable = generate_InstanceVariable(df_meta)
#SubstantiveValueDomain = generate_SubstantiveValueDomain(df_meta)
SubstantiveConceptualDomain = generate_SubstantiveConceptualDomain(df_meta)
SentinelConceptualDomain = generate_SentinelConceptualDomain(df_meta)
ValueAndConceptDescription = generate_ValueAndConceptDescription(df_meta)
SubstantiveConceptScheme = generate_SubstantiveConceptScheme(df_meta)
SentinelConceptScheme = generate_SentinelConceptScheme(df_meta)
Concept = generate_Concept(df_meta)
#Codelist = generate_Codelist(df_meta)
#Code = generate_Code(df_meta)
LogicalRecord = generate_LogicalRecord(df_meta)
PhysicalDataset = generate_PhysicalDataset(df_meta, spssfile)
PhysicalRecordSegment = generate_PhysicalRecordSegment(df_meta)
PhysicalSegmentLayout = generate_PhysicalSegmentLayout(df_meta)
ValueMapping = generate_ValueMapping(df, df_meta)
ValueMappingPosition = generate_ValueMappingPosition(df_meta)
InstanceValue = generate_InstanceValue(df, df_meta)
DataPoint = generate_DataPoint(df, df_meta)
DataPointPosition = generate_DataPointPosition(df, df_meta)
DataStore = generate_DataStore(df_meta)
WideDataSet = generate_WideDataSet(df_meta)
WideDataStructure = generate_WideDataStructure(df_meta)
PrimaryKey = generate_PrimaryKey(df_meta)
PrimaryKeyComponent = generate_PrimaryKeyComponent(df_meta)
MeasureComponent = generate_MeasureComponent(df_meta)
IdentifierComponent = generate_IdentifierComponent(df_meta)


json_ld_graph = DataStore + PhysicalDataset + PhysicalRecordSegment + PhysicalSegmentLayout + ValueMapping + \
ValueMappingPosition + DataPoint + DataPointPosition + InstanceValue + LogicalRecord + WideDataSet + \
WideDataStructure + IdentifierComponent + MeasureComponent + PrimaryKey + PrimaryKeyComponent + InstanceVariable + \
SubstantiveConceptualDomain + SubstantiveConceptScheme + SentinelConceptualDomain + ValueAndConceptDescription + \
SentinelConceptScheme + Concept

SyntaxError: invalid syntax (3506403599.py, line 51)

## Create Json

In [ ]:
# Create a dictionary with the specified "@context" and "@graph" keys
json_ld_dict = {
    "@context": [
        "https://ddi-alliance.bitbucket.io/DDI-CDI/DDI-CDI_v1.0-rc1/encoding/json-ld/ddi-cdi.jsonld",
        {
            "skos": "http://www.w3.org/2004/02/skos/core#"
        }
    ],
    "@graph": json_ld_graph
}

def default_encode(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    elif pd.isna(obj):  # Checks for pd.NA
        return None
    elif isinstance(obj, pd.Timestamp):  # Checks for Timestamp
        return obj.isoformat()
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")



# Convert the Python dictionary to a JSON string with pretty formatting

json_ld_string = json.dumps(json_ld_dict, indent=4, default=default_encode)
# New code to write the JSON-LD string to a file
with open('output.jsonld', 'w') as file:
    file.write(json_ld_string)

##### 